## Random Forest
Random Forest is an ensemble learning algorithm that builds multiple decision trees and combines their outputs to improve accuracy and reduce overfitting. It is used for both classification and regression tasks.

It works by:

1. Creating multiple decision trees using different random subsets of the training data (Bootstrap Aggregation or Bagging).
2. Selecting a random subset of features at each split to ensure diversity among trees.
3. Aggregating the predictions from all trees (majority voting for classification, averaging for regression).

### Assumptions
1. The dataset has enough diversity so that multiple trees can learn different patterns.
2. Relationships between features and target variables can be captured through multiple decision boundaries.
3. More trees generally improve performance but come at a computational cost.

### Limitations
1. Computationally expensive – Training many trees requires more memory and time.
2. Less interpretable than a single decision tree.
3. Prone to overfitting with noisy data if the number of trees is too large.
4. Not ideal for extremely high-dimensional data (may require feature selection).
### When to use?
1. High accuracy – When improving predictive performance is the priority.
2. Handling missing data – Can work well even with missing values.
3. Large datasets – Works effectively with large datasets with many features.
4. Reducing overfitting – More robust than a single decision tree, making it suitable for complex problems.

In [2]:
# We will use the decision tree to create random forest classifier
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None,
                 left=None, right=None, * , value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None
    

class CustomDecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100,
                 n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None
        
    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))
        # cheking for the stopping criteria
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        feat_idxs = np.random.choice(n_feats,
                                     self.n_features,
                                     replace=False)
        # find the best fit
        best_feature, best_threshold = self._best_split(X, y, feat_idxs)

        # creating the child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)

        return Node(best_feature, best_threshold, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idxs, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # creating a children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted average entropy of children 
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the information gain
        information_gain = parent_entropy - child_entropy
        return information_gain   

    def _split(self, X_column, split_threshold):
        left_idxs = np.argwhere(X_column <= split_threshold).flatten()
        right_idxs = np.argwhere(X_column > split_threshold).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])

    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x,self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


In [ ]:
class CustomRandomForest:
    def __init__(self, n_trees=10, max_depth=10,
                 min_samples_split=2, n_features=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = CustomDecisionTree(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                n_features=self.n_features
            )
            X_sample, y_sample = self._bootstrap_samples(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def _bootstrap_samples(self):
        pass

    def _most_common_label(self):
        pass

    def predict(self):
        pass